In [30]:
import sys
import os
import glob
import pandas as pd
from collections import Counter
from sklearn.preprocessing import Imputer

import numpy as np
import sklearn
print (sklearn.__version__)

from sklearn import linear_model
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve, auc
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.svm import  SVC

from sklearn.externals import joblib

0.19.1


In [2]:
# ディレクトリ名
input_dir = ur"C:/Users/mirait/wk/forKaggle/input/"
output_dir = ur"C:/Users/mirait/wk/forKaggle/output/"

In [3]:
# 訓練データを読み込む
train_path = input_dir + ur"train.csv"
train_data = pd.read_csv(train_path, encoding="cp932",low_memory=False)
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# テストデータを読み込む
test_path = input_dir + ur"test.csv"
test_data = pd.read_csv(test_path, encoding="cp932",low_memory=False)
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
print "訓練データ：レコード長", len(train_data)
print "訓練データ：カラム数",len(train_data.columns)
print "テストデータ：レコード長", len(test_data)
print "テストデータ：カラム数",len(test_data.columns)

訓練データ：レコード長 891
訓練データ：カラム数 12
テストデータ：レコード長 418
テストデータ：カラム数 11


In [6]:
# 分析に生かすために項目の特徴量を取得
def feature(df):
    
    df_feature = pd.DataFrame()
    
    for i in (range(len(df.columns))):
        tmp = pd.DataFrame()
        tmp = df.iloc[:, [i]]
        
        selList = list()
        record_cn = len(tmp) #レコード数
        column_name = tmp.columns[0] #カラム名
        value_type_cn = len(pd.value_counts(tmp.values.flatten())) #値の種類数(NAはカウント外)
        NA_cn = tmp.isnull().sum().values[0] #NA件数

        feature_list = list([record_cn, column_name, value_type_cn, NA_cn])
        now_column = pd.DataFrame(feature_list).T
        #print res1
        
        df_feature = pd.concat([df_feature, now_column], ignore_index=True)
    
    df_feature.columns = [u'レコード数', u'カラム名', 'v_count', u'NA件数']
        
    col_names = list(df_feature.columns)
    df_feature = df_feature.loc[:, col_names]

    return df_feature

In [7]:
# 訓練データの特徴量
feature_train = feature(train_data)
print len(feature_train)

12


In [8]:
# テストデータの特徴量
feature_test = feature(test_data)
print len(feature_test)

11


In [9]:
output_feature_train = os.path.join(output_dir, "column_feature_train.csv")
feature_train.to_csv(output_feature_train, encoding="cp932",index=False)

In [10]:
output_feature_test = os.path.join(output_dir, "column_feature_test.csv")
feature_test.to_csv(output_feature_test, encoding="cp932",index=False)

In [11]:
# データ型が定義されているファイルを読み込み　（データ型は独自判断）
type_list = input_dir + ur"type_list.csv"
df_type = pd.read_csv(type_list, encoding="cp932", low_memory=False)

In [12]:
df_type

,column_name,type
0,PassengerId,char
1,Survived,char
2,Pclass,char
3,Name,char
4,Sex,char
5,Age,number
6,SibSp,number
7,Parch,number
8,Ticket,char
9,Fare,number


In [17]:
type_char = df_type.query('type == "char"')
char_type_column = set(list(type_char["column_name"]))
train_char = train_data.loc[:, char_type_column]

In [22]:
# 値の種類数が100以上の項目は除外対象とする　（カテゴリ項目のみ）
over_value100_list = list(feature_train.query("v_count >= 100")[u"カラム名"].values)

In [21]:
# カテゴリ項目を抽出し除外
char_column = list(train_char.columns)
wk_set = set(char_column) - set(over_value100_list)
wk_sel = list(wk_set)
clean_train = train_data.loc[:, wk_sel]

In [31]:
# 欠損値が存在する項目（Age、Embarked）は最頻値で置換
imp = Imputer(missing_values='NaN', strategy='median', axis=1, verbose=0, copy=True)
imp.fit(clean_train[clean_train.columns["Age"]])
df["Age"]=imp.transform(clean_train[df.clean_train["Age"]]).ravel()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
# 値の種類数が100以上の項目を抽出
over_value100_list = list(df_feature.query("v_count >= 100")[u"カラム名"].values)